## Median_search

In [16]:
## Returns all possible permutations of length l, with characters A,C,G,T
##
## Example: 
## Input:  length=2
## Output: ["AA", "AC", "AT", "AG", "CA", "CC", "CT", "CG", "TA", "TC", "TT", "TG", "GA", "GC", "GT", "GG"]
function allPermutations(length)
    
    if length==1
        return ["A","C","T","G"]
    end
    
    suffix_permutations=allPermutations(length-1)
    
    ans=[]
    for c in "ACTG"
       for suffix in suffix_permutations
            push!(ans,c*suffix)
        end
    end
    
    return ans
end

allPermutations (generic function with 1 method)

In [72]:
## Given two strings of equal length, returns the no. of mismatches characters
##
## Example: 
## Input: string1="ATG", string2="ACG"
## Output: 1
function getMismatches(string1,string2)
    mismatches = 0
    for (c1,c2) in zip(string1,string2)
        if c1!=c2
            mismatches+=1
        end
    end

    return mismatches
end

getMismatches (generic function with 1 method)

In [75]:
## dna - all dna strings in an array
## l   - length of motif
##
## Returns the best motif, score for the best motif, and the start positions for the motif
function medianString(dna,l)
    
    t = length(dna)    ##  No. of dna sequences
    n = length(dna[1]) ##  length of each dna sequence
   
    scores = [l for i in 1:t]
    start =  [1 for i in 1:t]
    
    best_score = l*t 
    best_positions = []
    
    all_permutations = allPermutations(l)
    median_string = all_permutations[1]
    
    for permutation in all_permutations

        scores = [l for i in 1:t]
        start =  [1 for i in 1:t]

        for j in 1:t   
            dna_seq = dna[j]
            for start_position in 1:n-l+1

                mismatches = getMismatches(permutation,dna_seq[start_position:start_position+l-1])

                if scores[j]>mismatches
                    scores[j] = mismatches
                    start[j] = start_position
                end
            end
        end

        curr_score = sum(scores)
        if curr_score<best_score
             best_score = curr_score
             best_positions = deepcopy(start)
             median_string = permutation
        end
    end
    
    return median_string,best_score,best_positions
end

medianString (generic function with 2 methods)

In [76]:
position,score,permutation = medianString(["TGGCGTCG","GGCGGGCG","TCTTCGCC","CCCGCTCC"],3)

("CGC", 2, [2, 1, 5, 3])

## Branch and Bound

In [90]:
## dna - all dna strings in an array
## l   - length of motif
##
## Returns the best motif, score for the best motif, and the start positions for the motif
function branch_and_bound_with_median_search(dna,l,verbose)
    
    t = length(dna)    ##  No. of dna sequences
    n = length(dna[1]) ##  length of each dna sequence
   
    scores = [l for i in 1:t]
    start =  [1 for i in 1:t]
    
    best_score = l*t 
    best_positions = []
    
    all_permutations = allPermutations(l)
    median_string = all_permutations[1]
    
    for permutation in all_permutations

        scores = [l for i in 1:t]
        start =  [1 for i in 1:t]
        
        current_score = 0
        for j in 1:t  
            dna_seq = dna[j]
            for start_position in 1:n-l+1
                mismatches = getMismatches(permutation,dna_seq[start_position:start_position+l-1])
                if scores[j]>mismatches
                    scores[j] = mismatches
                    start[j] = start_position
                end
            end
            current_score+=scores[j]
            if current_score>=best_score
                if verbose==true
                    println("Branching as initial results are bad for ",permutation," after ",j," sequences.")
                end
                break
            end
        end

        curr_score = sum(scores)
        if curr_score<best_score
             best_score = curr_score
             best_positions = deepcopy(start)
             median_string = permutation
        end

    end
    
    return median_string,best_score,best_positions
end

branch_and_bound_with_median_search (generic function with 2 methods)

In [91]:
position,score,permutation = branch_and_bound_with_median_search(["TGGCGTCG","GGCGGGCG","TCTTCGCC","CCCGCTCC"],3,false)

("CGC", 2, [2, 1, 5, 3])

## Genetic Algorithm

In [22]:
#import Pkg 
#Pkg.add("Distributions")
#include("common.jl")
using Distributions

In [21]:
function generate_random_potential_motif(motif_length)
    motif=sample( ["a","c","t","g"] , motif_length; replace=true, ordered=false)
    motif=join(motif)
    return motif
end

function calculate_hamming_distance(string, motif)
    error = 0
    for i in 1:length(motif)
        if string[i] != motif[i]
            error += 1
        end
    end
    return error
end

function mutate_and_generate_adjacent_motif(motif, all_created_motifs)
    
    this_motif_characters = split(motif, "")
    random_index = rand(1:length(motif))
    random_char = rand(motif_characters)
    
    while motif[random_index] == random_char
        random_char = rand(motif_characters)
    end
    
    this_motif_characters[random_index] = random_char
    
    generated_motif = join(this_motif_characters)
    
    if generated_motif in all_created_motifs
        return mutate_and_generate_adjacent_motif(motif,all_created_motifs)
    end
    
    return generated_motif
end

function get_all_substrings(string, l)
    substrings = []
    for start in 1:length(string)-l+1
        push!(substrings, string[start:start+l-1])
    end
    return substrings
end


function validate_by_hamming_distance(string, motif, goal )

    string_motifs = get_all_substrings(string, length(motif))
    for curr_motif in string_motifs
        error =  calculate_hamming_distance(curr_motif, motif)
        if error <= goal
            return true
        end
    end
    return false
end


function is_motif_valid(input_strings, motif, goal=1 )
    for string in input_strings
        if validate_by_hamming_distance(string, motif, goal) == false
            return false
        end
    end
    return true
end

function calculate_minimum_hamming_distance(motif, string)
    substrings=get_all_substrings(string, length(motif))
    return minimum([calculate_hamming_distance(str, motif) for str in  substrings])
end

function calculate_fitness_score(input_strings, motif)
    fitness_score = 0
    for string in input_strings
        fitness_score += calculate_minimum_hamming_distance(motif, string)
    end
    return (len(motif) * length(input_strings)) - fitness_score
end

is_motif_valid (generic function with 3 methods)

In [7]:
function reproduce(mom, dad, l)
    random_index = rand(1:l)
    son = mom[1:random_index] + dad[random_index+1:end]
    daughter = dad[1:random_index] + mom[random_index+1:end]
    return (son, daughter)
end

function generate_initial_population(all_created_motifs,motif_len,size)
    population = []
    for i in 1:size
        motif = generate_random_potential_motif(motif_len)
        push!(population, motif)
        push!(all_created_motifs, motif)
    end
    return collect(Set(population))
end

function get_random_from_population(population, partner=[])
    random_index = rand(1:length(population))
    motif = population[random_index]
    if ((!isempty(partner)) && (partner == motif))
        return get_random_from_population(population, partner)
    else
        return motif, get_fitness_score_percentage(motif)
    end
end

function get_fitness_score_percentage(input_strings,motif,n)
    return calculate_fitness_score(input_strings,motif) / (length(motif) * n)
end

generate_initial_population (generic function with 1 method)

In [ ]:
function genetic_algorithm(input_strings,motif_len,size,verbose=false)
    n=length(input_strings[0])
    time_start = time()
    done = false
    answer = ""
    most_close_to_answer = ""
    most_close_to_answer_score = 0
    all_created_motifs=[]
    population = generate_initial_population(all_created_motifs,size)
    while done==false
        new_population = []
        while length(population) < size
            push!(population, generate_random_potential_motif(motif_len))
        end
        for i in 1:length(population)
            #sleep(0.1)
            (random_motif_dad, dad_fitness_score) = get_random_from_population(population, [])
            (best_motif, best_score) = (random_motif_dad, dad_fitness_score)
            (random_motif_mom, mom_fitness_score) = get_random_from_population(population, random_motif_dad)
            if mom_fitness_score > best_score
                (best_motif, best_score) = (random_motif_mom, mom_fitness_score)
            end
            (motif_son, motif_daughter) = reproduce(random_motif_mom, random_motif_dad, l)
            if verbose==true
                print("\n", motif_son, " is child of ", random_motif_mom, " and ", random_motif_dad, "!")
                print("\n", motif_daughter, " is child of ", random_motif_mom, " and ", random_motif_dad, '!')
            end
            random_probability = rand(Uniform(0, 1))
            son_score = get_fitness_score_percentage(input_strings,motif_son,n)
            if son_score > best_score
                (best_motif, best_score) = (motif_son, son_score)
            else random_probability < 0.1
                motif_son = mutate_and_generate_adjacent_motif(motif_son, all_created_motifs)
                son_score = get_fitness_score_percentage(input_strings,motif_son,n)
                if son_score > best_score
                    (best_motif, best_score) = (motif_son, son_score)
                end
            end

            daughter_score = get_fitness_score_percentage(motif_daughter)
            if daughter_score > best_score
                (best_motif, best_score) = (motif_daughter, daughter_score)
            else random_probability < 0.1
                motif_daughter = mutate_and_generate_adjacent_motif(motif_daughter, all_created_motifs)
                daughter_score = get_fitness_score_percentage(input_strings,motif_daughter,n)
                if daughter_score > best_score
                    (best_motif, best_score) = (motif_daughter, daughter_score)
                end
            end

            if son_score > daughter_score
                push!(new_population, motif_son)
            else
                push!(new_population, motif_daughter)
            end

            if (isempty(most_close_to_answer) || best_score > most_close_to_answer_score)
                (most_close_to_answer, most_close_to_answer_score) = (best_motif, best_score)
            end
            print(best_motif, " is best motif with fitness of ", string(get_fitness_score_percentage(best_motif)))
            push!(new_population, best_motif)
        end
        population = collect(Set(new_population))

        if time() - ga_time > enough_time
            done = True
        end
        for motif in population
            if is_motif_valid(motif) == true
                done = true
                answer = motif
                break
            end
        end
    end
    
    if !isempty(answer)
        return answer
    else
        return Dict("best_answer"=>most_close_to_answer, "best_score"=>most_close_to_answer_score)
    end
end

st = time()
answer = implement_genetic_algorithm(50)
if string(answer) == answer
    print("\n", answer, " is a valid motif.")
else
    print("\nBest found motif is ", answer["best_answer"], " with fitness score of ", string(answer["best_score"]))
end
elapsed = time() - st

In [ ]:
input_strings = [
    "AAAATTATTCTTCCTTCGCTTTGTTTTTAGACATAATGTTAAATTTATTTTGAAATTTAAAGCAACATAAAAGAACATGTGATTTTTCTACTTATTGAAAGAGAGAAAGGAAAAAAATATGAAACAGGGATGGAAAGAATCCTATGCCTGGTGAAGGTCAAGGGTTCTCATAACCTACAGAGAATTTGGGGTCAGCCTGTCCTATTGTATATTATGGCAAAGATAATCATCATCTCATTTGGGTCCATTTTCCTCTCCATCTCTGCTTAACTGAAGATCCCATGAGATATACTCACACTGAATCTAAATAGCCTATCTCAGGGCTTGAATCACATGTGGGCCACAGCAGGAATGGGAACATGGAATTTCTAAGTCCTATCTTACTTGTTATTGTTGCTATGTCTTTTTCTTAGTTTGCATCTGAGGCAACATCAGCTTTTTCAGACAGAATGGCTTTGGAATAGTAAAAAAGACACAGAAGCCCTAAAATATGTATGTATGTATATGTGTGTGTGCATGCGTGAGTACTTGTGTGTAAATTTTTCATTATCTATAGGTAAAAGCACACTTGGAATTAGCAATAGATGCAATTTGGGACTTAACTCTTTCAGTATGTCTTATTTCTAAGCAAAGTATTTAGTTTGGTTAGTAATTACTAAACACTGAGAACTAAATTGCAAACACCAAGAACTAAAATGTTCAAGTGGGAAATTACAGTTAAATACCATGGTAATGAATAAAAGGTACAAATCGTTTAAACTCTTATGTAAAATTTGATAAGATGTTTTACACAACTTTAATACATTGACAAGGTCTTGTGGAGAAAACAGTTCCAGATGGTAAATATACACAAGGGATTTAGTCAAACAATTTTTTGGCAAGAATATTATGAATTTTGTAATCGGTTGGCAGCCAATGAAATACAAAGATGAGTCTAGTTAATAATCTACAATTATTGGTTAAAGAAGTATATTAGTGCTAATTTCCCTCCGTTTGTCCT",
    "CAGCTTTGCGCCTCCACTGTCACCCTCAGGAATGTTCCACATACTCAGCGAGTATGCTTGGGGGGCAAAAGGGTGAAAGATACAAAAGCTTCTGATATCTATTTAACTGATTTCACCCAAATGCTTTGAACCTGGGAATGTACCTCTCCCCCTCCCCCACCCCCAACAGGAGTGAGACAAGGGCCAGGGCTATTGCCCCTGCTGACTCAATATTGGCTAATCACTGCCTAGAACTGATAAGGTGATCAAATGACCAGGTGCCTTCAACCTTTACCCTGGTAGAAGCCTCTTATTCACCTCTTTTCCTGCCAGAGCCCTCCATTGGGAGGGGACGGGCGGAAGCTGTTTTCTGAATTTGTTTTACTGGGGGTAGGGTATGTTCAGTGATCAGCATCCAGGTCATTCTGGGCTCTCCTGTTTTCTCCCCGTCTCATTACACATTAACTCAAAAACGGACAAGATCATTTACACTTGCCCTCTTACCCGACCCTCATTCCCCTAACCCCCATAGCCCTCAACCCTGTCCCTGATTTCAATTCCTTTCTCCTTTCTTCTGCTCCCCAATATCTCTCTGCCAAGTTGCAGTAAAGTGGGATAAGGTTGAGAGATGAGATCTACCCATAATGGAATAAAGACACCATGAGCTTTCCATGGTATGATGGGTTGATGGTATTCCATGGGTTGATATGTCAGAGCTTTCCAGAGAAATAACTTGGAATCCTGCTTCCTGTTGCACTCAAGTCCAAGGACCTCAGATCTCAAAAGAATGAACCTCAAATATACCTGAAGTGTACCCCCTTAGCCTCCACTAAGAGCTGTACCCCCTGCCTCTCACCCCATCACCATGAGTCTTCCATGTGCTTGTCCTCTCCTCCCCCATTTCTCCAACTTGTTTATCCTCACATAATCCCTGCCCCACTGGGCCCATCCATAGTCCCTGTCACCTGACAGGGGGTGGGTAAACAGACAGGTATATAGCCCCTTCCTCTCCAGCCAGGGC",
    "TTGACAGACAGTGTGGAGGGATTACTTGAATCTTGTGAATAGAGGAAAGAGTAGAATCAGATTATCCTGACTCCTGCCTGAAGCTTTACATATTCAGAGAAAAATGTTGGAAGAAACTTTGATATAATGCTATGTCTGTGATCAGGCACACATTTTACTGGACTTTTACTGTCAGGGCCGTCATTTAGTGCCAAGATGTCTAGAGAGTTCTTAATAAGTGTACTCAATTGGCTGAGAAAATGTGTCCATGCAAAAAACCAAACACCGCGTGTTCTCACTCATAGATGGGAATTGAACAATGAGAATACTTGGACACAGGAAGGGGAACATCACACTCTGGGGACTGTTGTGGGGTGGGGGGAGGGGGGAGGGATAGCATTAGAAGATATACCTAATGCTAAATGATGAGTTAATGGGTGCAGCACACCAGCATGGCACATGTATACATATGTAACTAACCTGCACATTGTGCACATGTACCCTAAAACTTAAAGTATAATAATAATAAAAAAATGTGTCCATGGCTCTGGGAGGAGCATGTTTGTTTTCCTCATTTCCCAGTCTGTAAATAAGCAAATTGAAAGGGGTTAGTGATAATGTCCATCTCCAGAAGCTGTCAGATTTCCTTTGTCAAACTCTATGATTTGGGCTGAAGTAGGTGTTGGAGAGGCAGCTACCACGTGCACCCAGATGGCCACTCGTTTAATATGTTACCATTTCCCATTATTTTCGCAGGATAGATAGCCAAAGTGGAGCCCTGAGAGATTTCTTCATTTTTCCTGTCATAAAGAATTGGTAATTCAGTAGTCATAGGAGTTTGTAATAAATAACTCACATTGATTTCTCTGTTCTGAAATAATTTTGCTTCCCCTCTTCCCGAAGCTCTGACACCTGCCCCAACAAGCAATGTTGGAAAATTATTTACATAGTGGCGCAAACTCCCTTACTGCTTTGGATATAAATCCAGGCAGGAGGAGGTAGCTCTAAGGCAAGAGATCTA",
    "ATACACGTACTACACATTGGACTCTTGGGTAGTCTCTAGGGCTGTAGCAAGGAGCCTTGCTCCCAAGGGACTCATTTACACAATCCTGTGAACGGACCAAGAGTAAACAGTGTGCTCAATGCTGTGCCTACGTGTGTTAGCCCACGCGGCCAGCCTGAGGAGTCAGGGAAGGCTCCCCTAGGCAAAGCCCCCAACCAGAATCAAGTCTTAATGGTTAAAGAGCTCCATCACCCAAAAAGGATTGAGGGCCTACCTTCAACTGAACAGCTAATGCATAATCTCAGAAACTGTGAGTCAAAATTCCCTGGAATAACTCCACTTTATCCCCAATCTCCTTGCCACCTAGACCAAGGTCCATTCACCACCCTGTCCCCAGCACTGACTGCACTGCTGTGGCCACACTAAAGCTTGGCTCAAGACGGAGGAGGAGTGAGGAAGCTGCTGCACCAATATGGCTGGTTGAGGCCGCCCAAGGTCCTAGAAGGAGGAAGTGGGTAAATGCCATATCCAAAAAGATACAGAAGCCTCAGGTTTTATCGGGGGCAGCAGCTTCCTTCTCCTTCCCCGACCTGTGGCCAAGTCACAAAGCACCACAGCTGTACAGCCAGATGGGGGAAGGGAGGAGATTAGAACTGTAGGCTAGAGTAGACAAGTATGGACCAGTTCACAATCACGCTATCCCAAGCAGAAAGTGATGGTGGCTTGGACTAGCACGGTGGTAGTAGAGATGGGGTAAAGATTCAAGAGACATCATTGATAGGCAGAACCAATAGGACATGGTAATAAACTATTCTCAGGAAAGGGGAGGAGTCATGGCTTTCAGCCATGAGCATCCACCCTCTGGGTGGCCTCACCCACTTCCTGGCAATTCTAGCCACCATGAGTCCAGGGGCTATAGCCCTTTGCTCTGCCCGTTGCTCAGCAAGTTACTTGGGGTTCCAGTTTGATAAGAAAAGACTTCCTGTGGAGGAATCTGAAGGGAAGGAGGAGGAGCTGGCCC",
    "TGGGAGGCAAAGATGGTGGCAGGTGGGTGGGAAAATGTGGGGTTTGGGGGTCACAGGAGAGACCAAGGGGCAGGCCTGGAGCATGGCACGCTCCCTGATCACAGCTCTCCACTGGAAGGCCATGGAGGGAGTGATGGGGAGAGCCTGCAACTCCTGGAGATAAGCCTGATTCTGACTTTCTTTGAAGGATTTTTTTCTTTCGACTTCTTTCCTGTGAAAGAGAATATCCGAGAAATGAGAATTTTAACTAAAGAGAAAACCTTTGCATTTGCTACTGATAAATATCTGGCCTATGCGGGAAAGTCATGGGTGCCCTGGGATTCTGGCTCCACTGCTAACAAGACCTTGTTAGACCTTCTTAACTGCCCCTCCAGAGAGGAGAAAGGCTCTTGGAGGTAGAGTGAGATTGGGGAGCTTGGCAGAGCCCTAGGATGAACTGTGTTTTTGTTTTGTTTTGTTGCAGGTTGGAGCCTTTTTCCTGCAGAAAGAGAAGCAATTCCGGGCTTTTCCACAGATAACCTGCTAGGGCTCGCAGAGACCTGGATTCTAACCCCATCTCGCCTTTCCTCACCTGGGGCATTGTTTTCTGCGTTTGAGAACTGCTGGGTGGGCTGGTGTTACCCAGGATTTTTAAATTTTTGCAGGAGACGCTTTGCAGGAGATCTCTAGGGCTTTTTTCTGTGTCAATTAAAGAGCCTGGCCAGCTGGACCTGGGCTGTCTTTTTGACAAAAACAAACGTCATCCCCCTCCCAGCTGAGCACTTGTTAGAACTGGACTTTAACTGAGGGCCTGAACCCCCTAACAACGGGACAAACAGTATGAAATCAAAACCGTTTACCCTCCTCCCACCAGCGGTTTGCGTAGGGCCTTGGGTGCACTAGCAAAACAAACTTATTTTGAACACTCAGCTCCTAGCGTGCGGCGCTGCCAATCATTAACCTCCTGGTGCAAGTGGCGCGGCCTGTGCCCTTTATAAGGTGCGCGCTGTGTCCAGCGAGC",
    "TTACAACAATCCATGCATACTTTTTTTAAATTTTCTTTTTAGACTCATATTTAGTTTGCTTCAAAAACTAATTGCTCTACTTTATGCAAACACATTATTTTAATATATATTTTGTTGAAAAGATAGTATCCAGATGGTATGCAAGGCTTAGACTCTGACAATTTCAGAGTCAAATTGGGGTCAATAATAATTAAGCAATATTAGCTCATATAATTCTTTTAGTGATTTTATAGGTAGTATTTTGCCCACAATTTCTCAGGTAAGCATAGTACAGCCAGCATTTATATCCAAATTTATTATATCAAATCCATTGCTTTTAAACATTATGCTAGCTTAAACCAGTAGTGTAATTAATTTTATACATTTTACTACATGAAATGAGAAGGTTTACAATATGATTTTTTATAATTCAATGAGTGCTATCTGTGGTAAATAAATAGATAAAAATGTTGATTTTATCTCTAAATTACTGATCTGGGCACCTCTATTTTATGTTTTATACTTAAGGATTTGGTCACTTTTTCAACCTTAAGATCAAAGACAAGATACAGGGAATAATTTTTATTTAATCATTACTTTTTATAAAATATTATCTTCTAATTGTTATATCCTGAAAATTTTTCACAGATTCTGTCTTTCAAATATGTTTCTACTCTCCAGTGAGGATCTAAATTGCACACAATGGACAAAATAGTTATTATAACAAAATCCAAATATAAAATAAATATAATTTTGAAAAGAGAAAAAATAGAAAGAACTACTTTTATGTTCTTAAGATATTAATAGTTCACAGCTTTGAGAAATCAAAGAGTATCTGACAGTACAATTACTAATTAACTTAGATTCCCAGAGAGAAACATTTATGTAAACTACTTTCAGGGTTAAGGCTTTTAACAATTGTATGTTGAGCAGAAGATTATTAAACACTGATAGGCTGGTAAGGGTGCAATAAAACTTTATGAGTAGGTCAATATATACCTAAGCAGCTTTCCTTGATCAT"
]

## Hill Climbing

In [ ]:
maximum_restarts = 10

In [ ]:
function calculate_fitness_score(input_strings, motif)
    fitness_score = 0
    valid = true
    for string in input_strings
        score = calculate_minimum_hamming_distance(motif, string)
        if score > 1
            valid = false
        end
        fitness_score += score
    end
    return (fitness_score, valid)
end

In [ ]:
function check_motif_fitness_score_and_validate(motif,maximum_restarts)
    motif_is_valid_for_all_strings_inputs = true
    (this_motif_fitness_score, is_motif_valid) = calculate_fitness_score(input_strings, motif)
    next_motif = motif
    for i in 1:maximum_restarts
        adjacent_motif = generate_adjacent_motif(motif)
        (adjacent_motif_fitness_score, adjacent_valid) = calculate_fitness_score(adjacent_motif)
        if verbose==true
            println("Motif: ",motif)
            println("Adjacent_motif: ",adjacent_motif)
            println("Adjecent motif fitness score: ", adjacent_motif_fitness_score)
        end
        
        if (adjacent_motif_fitness_score < this_motif_fitness_score)
            println(Adjacent_motif, " is better than ", next_motif, " with ", string(this_motif_fitness_score - adjacent_motif_fitness_score), " scores.")
            next_motif = adjacent_motif
            break
        end
    end
    if (next_motif == motif)
        return (motif, this_motif_fitness_score, is_motif_valid)
    else
        print("Going to adjacent motif.\n\n")
        return check_motif_fitness_score_and_validate(next_motif,maximum_restarts)
    end
end

In [ ]:
function start_hill_climbing(input_strings,maximum_restarts)
    answers = []
    for i in 1:maximum_restarts
        (motif, fitness, is_valid) = check_motif_fitness_score_and_validate(generate_random_potential_motif())
        if is_valid == true
            if isequal(motif in answers, false)
                push!(answers,motif)
                println(motif, " is a discovered motif with score of ", map_fitness_to_number(fitness))
            end
        else
            println(motif, " is not a motif because of ", map_fitness_to_number(fitness), " fitness score.")
        end
    end
    
    if length(answers) > 0
        println(string(length(answers)), " motif(s) with length of ", string(input_motif_length), " motifs were discovered.")
    else
        println("No motifs with length of ", string(input_motif_length), " were found.")
    end
end

In [ ]:
function map_fitness_to_number(fitness)
    return string(trunc(Int, ((input_motif_length * length(input_strings)) - (fitness / (length(input_strings) + 1)))))
end

In [ ]:
# st = start_time()
start_hill_climbing()
# end_time(st)